# Installation

To install the stable release version of pycortex, do the following:

In [41]:
# First, install some required dependencies
!pip install setuptools wheel numpy cython

In [42]:
# Install the latest release of pycortex from pip
!pip install pycortex

## Basic Configuration

To test if your install went well, you can run the pycortex

In [1]:
import cortex

Pycortex will automatically create a database filestore when it is first installed. 

You can check the location of the filestore after installing by running:

In [44]:
cortex.database.default_filestore

'/Users/freez1/pycortex/filestore/db'

If you want to move the filestore, you need to update the user configuration file.

you can check the location of the configuration file by running:

In [45]:
cortex.options.usercfg

'/Users/freez1/Library/Application Support/pycortex/options.cfg'

## Surface Database

Pycortex creates and maintains a simple flat-file database store all the data required to plot data on a cortical sheet.

you can download the MNI surface database from : https://github.com/eigenz1/Pycortex_tutorial

## Surfaces

Inside the filestore, surface names are formatted as {type}_{hemisphere}.{format}. Surfaces generally have three variables associated:

- Subject : a unique subject identifier (MNI in our tutorial)

- Type : the identifier for the type of geometry (fiducial, inflated, or flat)

- Hemisphere : the brain hemisphere of the surface, (lh or rh)

In [39]:
!tree /Users/freez1/pycortex/filestore/db/MNI/surfaces/

/Users/freez1/pycortex/filestore/db/MNI/surfaces/
├── flat_lh.gii
├── flat_rh.gii
├── inflated_lh.gii
├── inflated_rh.gii
├── pia_lh.gii
├── pia_rh.gii
├── wm_lh.gii
└── wm_rh.gii

0 directories, 8 files


In [40]:
# Selecting “surfaces” gives you a list of all surface types associated with that subject(MNI).
cortex.db.MNI.surfaces

Surfaces: [flat, wm, pia, inflated]

## Transform

Transformations in pycortex are stored as affine matrices, as defined in the Nifti headers.

Each transform is stored in its own subdirectory containing two files

In [3]:
!tree /Users/freez1/pycortex/filestore/db/MNI/transforms

/Users/freez1/pycortex/filestore/db/MNI/transforms
└── tutorial
    ├── matrices.xfm
    └── reference.nii.gz

1 directory, 2 files


# Visualization

In [4]:
import cortex
import numpy as np
import nibabel as nib

In order for following codes to run, you should have a flatmap for this subject in the pycortex filestore.

FYI, the reference image for this tutorial transform has shape of (79, 95, 79).

if your 3d image has shape of (91, 109, 91), you have to resample that image to shape of (79, 95, 79) as same as the reference for transform

## 1. Ramdom data

In [23]:
reference = nib.load("/Users/freez1/pycortex/filestore/db/MNI/transforms/tutorial/reference.nii.gz")
print(reference.shape)

(79, 95, 79)


In [5]:
subject = 'MNI'
transform = 'tutorial'

# Creating a random dataset that is the same shape as reference for transform
test_data = np.random.randn(79, 95, 79)

*Volume* module creates a Volume object for our test dataset for the given subject

In [25]:
# If you want to get thresholded data, 
# change the parameters vmin and vmax, then minimum and maximum values shown on the colorbar will be changed.
vol_data = cortex.Volume(test_data.transpose(2,1,0), subject, transform, cmap='J4', vmin=-2, vmax=2)

Stopping server


Creates a webGL MRI viewer running inside the current python process using the volume object.

In [26]:
cortex.webshow(vol_data)

Started server on port 51765


<JS: window.viewer>

Stopping server
Stopping server


If you have NaN values, those voxels show up transparent on the brain


In [28]:
test_data[10:15, :, :] = np.nan
vol_data = cortex.Volume(test_data.transpose(2,1,0), subject, transform, cmap='J4', vmin=-2, vmax=2)
cortex.webshow(vol_data)

Started server on port 30093


<JS: window.viewer>

Stopping server
Stopping server


## 2. Real data

Instead of the random test data, you can replace this with any numpy array of the correct dimensionality (79, 95, 79)

In [6]:
import nibabel as nib
pain_map = nib.load('pain_mega_re.nii').get_fdata() #3d array 
print(pain_map.shape)
vol_data = cortex.Volume(pain_map.transpose(2,1,0), subject, transform,)

(79, 95, 79)


In [7]:
cortex.webshow(vol_data)

Started server on port 44559


<JS: window.viewer>

## Save views (png format)

In [35]:
cortex.export.save_3d_views(vol_data,
                            base_name='tutorial_pain', 
                            list_angles=['top','back', 'medial_pivot', 'lateral_pivot'], #medial_pivot or lateral_pivot
                            list_surfaces=['inflated','inflated', 'inflated', 'inflated'],# 'inflated', 'flatmap', 'fiducial', 'inflated_cut',
                            size=(1024 * 4, 768 * 2),
                            trim=True, 
                            sleep=10)

Started server on port 40656
{'camera.azimuth': 180, 'camera.altitude': 0, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 0.5, 'surface.{subject}.pivot': 0, 'surface.{subject}.shift': 0, 'surface.{subject}.specularity': 0}
waiting for camera.altitude 0.1 -> 0
{'camera.azimuth': 180, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 0.5, 'surface.{subject}.pivot': 0, 'surface.{subject}.shift': 0, 'surface.{subject}.specularity': 0}
{'camera.azimuth': 0, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 0.5, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 10, 'surface.{subject}.specularity': 0}
{'camera.azimuth': 180, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 0.5, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 10, 'surface.{subject}.specularity': 0}
Stopping server


['tutorial_pain_top_inflated.png',
 'tutorial_pain_back_inflated.png',
 'tutorial_pain_medial_pivot_inflated.png',
 'tutorial_pain_lateral_pivot_inflated.png']

Stopping server


In [11]:
cortex.export.save_3d_views(vol_data,
                            base_name='tutorial_pain', 
                            list_surfaces=['flatmap'],# 'inflated', 'flatmap', 'fiducial', 'inflated_cut',
                            size=(1024 * 4, 768 * 2),
                            trim=True, 
                            sleep=10)

Started server on port 53526
{'camera.azimuth': 180, 'camera.altitude': 0, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 0, 'surface.{subject}.specularity': 0}
waiting for camera.altitude 0.1 -> 0
Stopping server


['tutorial_pain_flatmap_flatmap.png']

Stopping server


# Citation

If you use pycortex in published work, please cite the pycortex paper:

*Gao JS, Huth AG, Lescroart MD and Gallant JL (2015) Pycortex: an interactive surface visualizer for fMRI. Front. Neuroinform. 9:23. doi: 10.3389/fninf.2015.00023*